# Compatibilizador de malhas censitárias
---

### Importações

In [113]:
import warnings
warnings.simplefilter(action='ignore')

import os
import geopandas as gpd
import pandas as pd
import networkx as nx
from shapely import LineString, Polygon, MultiPolygon, distance, intersects, minimum_bounding_radius as min_radius
from shapely.geometry import box
from shapely.wkt import loads, dumps

In [114]:
### Célula para conectar com Google Drive
from google.colab import drive
drive.mount('/content/drive')

if not os.getcwd().endswith('Censo IBGE 2022/Compatibilização'):
    os.chdir('/content/drive/Shareddrives/SIG LabCidade/projetos/Censo IBGE 2022/Compatibilização')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [115]:
nome_compat = '2000-2010-RMSP-RMBS-AUJ'
nome_A = '2000'
nome_B = '2010'

In [116]:
if not os.path.isdir(nome_compat):
    os.mkdir(nome_compat)

### Funções gerais

In [117]:
UTMCODES = {
    '17S':"EPSG:31977",
    '18S':"EPSG:31978",
    '19S':"EPSG:31979",
    '20S':"EPSG:31980",
    '21S':"EPSG:31981",
    '22S':"EPSG:31982",
    '23S':"EPSG:31983",
    '24S':"EPSG:31984",
    '25S':"EPSG:31985",
    '17N':"EPSG:31971",
    '18N':"EPSG:31972",
    '19N':"EPSG:31973",
    '20N':"EPSG:31974",
    '21N':"EPSG:31975",
    '22N':"EPSG:31976",
    '23N':"EPSG:6210",
    '24N':"EPSG:6211"
    }

# Função para identificar projeção local UTM
def find_utm_proj(X, Y):
    # Hemisfério
    h = 'N' if Y > 0 else 'S'

    # Fuso
    if  -84 <= X < -78:
        f = '17'
    elif  -78 <= X < -72:
        f = '18'
    elif  -72 <= X < -66:
        f = '19'
    elif  -66 <= X < -60:
        f = '20'
    elif  -60 <= X < -54:
        f = '21'
    elif  -54 <= X < -48:
        f = '22'
    elif  -48 <= X < -42:
        f = '23'
    elif  -42 <= X < -36:
        f = '24'
    elif  -36 <= X < -30:
        f = '25'
    else:
        f=''

    fuse = f'{f}{h}'
    return UTMCODES[fuse]

In [118]:
# Função/regra para avaliar se geometria deve ser considerada expúria
def geomNotEspuria(geom):
    ap_ratio = geom.area/geom.length
    return (ap_ratio > 0.5 or geom.area > 500)

### Seleção de malhas

In [119]:
### Seleção para teste
muns = ['3503901', '3505708','3506359','3506607','3508405','3509007','3509205','3509601','3510609','3513009','3513504','3513801','3515004','3515103','3515707','3516309','3516408','3518305','3518701','3518800','3522109','3522208','3522505','3523107','3524006','3525003','3525201','3525904','3526209','3527306','3528502','3529401','3530607','3531100','3534401','3537602','3539103','3539806','3541000','3543303','3544103','3545001','3546801','3547304','3547809','3548500','3548708','3548807','3549953','3550308','3551009','3552502','3552809','3556453','3556503']

In [120]:
gdf_A = gpd.read_file('Setores IBGE.gpkg', layer='SP_2000')
gdf_A = gdf_A[gdf_A['CD_GEOCODI'].apply(lambda x: str(x)[:7] in muns)]

In [121]:
gdf_B = gpd.read_file('Setores IBGE.gpkg', layer='SP_2010')
gdf_B = gdf_B[gdf_B['CD_GEOCODI'].apply(lambda x: str(x)[:7] in muns)]

### Preparação dos geodataframes

In [122]:
# Tabular shapes
### Campo GROUP representa municípios
for gdf in [gdf_A, gdf_B]:
    gdf['geometry'] = gdf['geometry'].make_valid()
    gdf['GROUP'] = gdf['CD_GEOCODI'].apply(lambda x: str(x)[:11])

In [123]:
# Reprojetar camadas
Y = (gdf_B.total_bounds[1] + gdf_B.total_bounds[3])/2
X = (gdf_B.total_bounds[0] + gdf_B.total_bounds[2])/2
UTMCRS = find_utm_proj(X, Y)

gdf_A = gdf_A.to_crs(UTMCRS)
gdf_B = gdf_B.to_crs(UTMCRS)

## Grafo de vizinhança

In [124]:
# Listar vizinhos
def getNeighbors(row, geodataframe):
    df_neighbors = geodataframe.iloc[geodataframe['geometry'].sindex.query(row['geometry'], predicate='intersects')]
    df_neighbors = df_neighbors[df_neighbors['CD_GEOCODI'] != row['CD_GEOCODI']]
    if 'GROUP' in geodataframe.columns:
        group = row['GROUP']
        df_neighbors = df_neighbors.query('GROUP == @group')
    return df_neighbors

gdf_A['NEIGHBOR'] = gdf_A.apply(lambda x: getNeighbors(x, gdf_A)['CD_GEOCODI'].to_list(), axis=1)
gdf_B['NEIGHBOR'] = gdf_B.apply(lambda x: getNeighbors(x, gdf_B)['CD_GEOCODI'].to_list(), axis=1)

In [125]:
# Criar grafos de vizinhança
def makeGraph(gdf):
    G = nx.Graph()
    # Adicionar nós
    for i, row in gdf[['CD_GEOCODI', 'geometry']].iterrows():
        G.add_node(row['CD_GEOCODI'], center=row['geometry'].representative_point())
    # Dados das arestas
    dic = gdf[['CD_GEOCODI', 'NEIGHBOR']]\
          .explode('NEIGHBOR').dropna()\
          .to_dict(orient='records')
    # Adicionar arestas
    for i in dic:
        if (i['CD_GEOCODI'], i['NEIGHBOR']) not in G.edges:
            G.add_edge(i['CD_GEOCODI'], i['NEIGHBOR'],
                            geom=LineString([G.nodes[i['CD_GEOCODI']]['center'],
                                            G.nodes[i['NEIGHBOR']]['center']]))
    return G

graph_A = makeGraph(gdf_A)
graph_B = makeGraph(gdf_B)

### Exportação do grafo de vizinhança

In [126]:
# Geopackage de vizinhança
df_A = pd.DataFrame(graph_A.edges.data()).rename(columns={0:'node1', 1:'node2', 2:'geometry'})
df_A['geometry'] = df_A['geometry'].apply(lambda x: x['geom'])
geograph_A = gpd.GeoDataFrame(df_A, geometry='geometry', crs=UTMCRS)
geograph_A.to_file(f'{nome_compat}/grafo_vizinhanca.gpkg', layer=nome_A, driver='GPKG')

df_B = pd.DataFrame(graph_B.edges.data()).rename(columns={0:'node1', 1:'node2', 2:'geometry'})
df_B['geometry'] = df_B['geometry'].apply(lambda x: x['geom'])
geograph_B = gpd.GeoDataFrame(df_B, geometry='geometry', crs=UTMCRS)
geograph_B.to_file(f'{nome_compat}/grafo_vizinhanca.gpkg', layer=nome_B, driver='GPKG')

### Classificação das mudanças

In [127]:
def getNeighborhoods(geodataframe):
    dic = geodataframe[['CD_GEOCODI', 'NEIGHBOR']].to_dict(orient='records')
    dic = {k['CD_GEOCODI']:k['NEIGHBOR'] for k in dic}
    return dic

def getGeometries(geodataframe):
    geoms = geodataframe[['CD_GEOCODI', 'geometry']]
    geoms = geoms[['CD_GEOCODI', 'geometry']].to_dict(orient='records')
    geoms = {k['CD_GEOCODI']:k['geometry'] for k in geoms}
    return geoms

def bboxIntersects(geom1, geom2):
    bbox1 = box(*geom1.bounds)
    bbox2 = box(*geom2.bounds)
    return intersects(bbox1, bbox2)

def classificarMudancas(data_A, data_B):
    # Dicionários de relações
    dic_A = getNeighborhoods(data_A)
    dic_B = getNeighborhoods(data_B)

    # Dicionários de geometrias
    geoms_A = getGeometries(data_A)
    geoms_B = getGeometries(data_B)

    # Classificação
    class_data = {}
    for k, neighbors in dic_A.items():
        # Sem vizinhança
        if k not in dic_B:
            class_data[k] = 'Remoção'
    for k, neighbors in dic_B.items():
        # Distância acima da soma dos raios mínimos e BBoxes não sobrepostas
        if k in dic_A and k in dic_B \
            and distance(geoms_A[k].centroid, geoms_B[k].centroid) > (min_radius(geoms_A[k])+min_radius(geoms_B[k])) \
            and distance(geoms_A[k].centroid, geoms_B[k].centroid) >= 200 \
            and not bboxIntersects(geoms_A[k], geoms_B[k]):
            class_data[k] = 'Sobrescrito'
        # Vizinhança totalmente alterada sem vizinhos recém-criados, Código novo (em B)
        elif k not in dic_A and all([i in dic_A for i in neighbors]):
            class_data[k] = 'Inserção'
        # Vizinhança totalmente alterada com vizinhos recém-criados, Código novo (em B)
        elif k not in dic_A:
            class_data[k] = 'Criação'
        # Vizinhança mantida
        elif all([i in dic_A[k] for i in neighbors]) and all([i in neighbors for i in dic_A[k]]):
            class_data[k] = 'Manutenção'
        # Vizinhança parcialmente alterada sem vizinhos recém-criados
        elif any([i in dic_A[k] for i in neighbors]) and all([i in dic_A for i in neighbors]):
            class_data[k] = 'Ajuste'
        # Vizinhança parcialmente alterada com vizinhos recém-criados
        elif any([i in dic_A[k] for i in neighbors]):
            class_data[k] = 'Redesenho'
        # Vizinhança totalmente alterada sem vizinhos recém-criados, Código mantido (em B)
        elif not any([i in dic_A[k] for i in neighbors]) and all([i in dic_A for i in neighbors]):
            class_data[k] = 'Reposicionamento'
        # Vizinhança totalmente alterada com vizinhos recém-criados, Código mantido (em B)
        elif not any([i in dic_A for i in neighbors]):
            class_data[k] = 'Desassociação'
        # Outros casos (não identificados)
        else:
            class_data[k] = 'X'
    return class_data

alteracoes = classificarMudancas(gdf_A, gdf_B)

### Exportação dos setores classificados

In [128]:
gdf_A['CLASS'] = gdf_A['CD_GEOCODI'].apply(lambda x: alteracoes[x])
gdf_A[['CD_GEOCODI', 'GROUP', 'geometry', 'CLASS']].to_file(f'{nome_compat}/classificacao.gpkg', layer=nome_A, driver='GPKG')

In [129]:
gdf_B['CLASS'] = gdf_B['CD_GEOCODI'].apply(lambda x: alteracoes[x])
gdf_B[['CD_GEOCODI', 'GROUP', 'geometry', 'CLASS']].to_file(f'{nome_compat}/classificacao.gpkg', layer=nome_B, driver='GPKG')

## Grafo de compatibilização

In [152]:
G_compat = nx.Graph()

# Adicionar todos os nós
for i, row in gdf_A[['CD_GEOCODI', 'GROUP', 'CLASS', 'geometry']].iterrows():
    G_compat.add_node(f"A.{row['CD_GEOCODI']}",
                      malha='A',
                      nome=row['CD_GEOCODI'],
                      group=row['GROUP'],
                      geom=row['geometry'],
                      center=row['geometry'].representative_point(),
                      classe=row['CLASS'])
for i, row in gdf_B[['CD_GEOCODI', 'GROUP', 'CLASS', 'geometry']].iterrows():
    G_compat.add_node(f"B.{row['CD_GEOCODI']}",
                      malha='B',
                      nome=row['CD_GEOCODI'],
                      group=row['GROUP'],
                      geom=row['geometry'],
                      center=row['geometry'].representative_point(),
                      classe=row['CLASS'])

### Vínculo direto
Vincula setores que mantiveram os códigos e parte da estrutura de vizinhança entre A e B (Manutenção, Ajuste e Redesenho)

In [153]:
# Adicionar nós mantidos por Manutenção, Ajuste e Redesenho
### Estes nós permanecem com o mesmo código entre A e B
selected = ['Manutenção', 'Ajuste', 'Redesenho']
classif_A = gdf_A[['CD_GEOCODI', 'CLASS']].to_dict(orient='records')
classif_A = {k['CD_GEOCODI']:k['CLASS'] for k in classif_A}
for i, row in gdf_B.query('CLASS in @selected').iterrows():
    G_compat.add_edge(f"A.{row['CD_GEOCODI']}",
                      f"B.{row['CD_GEOCODI']}",
                      class_A=classif_A[row['CD_GEOCODI']],
                      class_B=row['CLASS'],
                      metodo='Vínculo direto')

### Análise de herança
Objetivo é buscar setores em B que tenham resultado de divisão de setores em A, e então vinculá-los.
Esta operação conceitualmente se dá em setores com Redesenho ou Desassociação em A (o setor já existia e foi fracionado ou teve vizinhança refeita) com setores com Desassociação, Sobrescrito, Insersão e Criação em B.

In [154]:
# Selecionar setores para análise
selected_class = ['Redesenho', 'Desassociação', 'Remoção']
alterados_A = gdf_A.query('CLASS in @selected_class')
selected_class = ['Sobrescrito', 'Desassociação', 'Inserção', 'Criação']
alterados_B = gdf_B.query('CLASS in @selected_class')

# Calcular áreas originais
alterados_A['AREA_ORIGINAL'] = alterados_A.area
alterados_B['AREA_ORIGINAL'] = alterados_B.area

# Renomear colunas
alterados_A = alterados_A.rename(columns={k:f'A.{k}' for k in alterados_A.columns})
alterados_A = alterados_A.set_geometry('A.geometry', crs=UTMCRS)
alterados_B = alterados_B.rename(columns={k:f'B.{k}' for k in alterados_B.columns})
alterados_B = alterados_B.set_geometry('B.geometry', crs=UTMCRS)

# Correção contra bug GEOSException: TopologyException: found non-noded intersection
alterados_A['A.geometry'] = [loads(dumps(geom, rounding_precision=3)) for geom in alterados_A['A.geometry']]
alterados_B['B.geometry'] = [loads(dumps(geom, rounding_precision=3)) for geom in alterados_B['B.geometry']]

# Interseção
intersecao = gpd.overlay(alterados_A, alterados_B, how='union')\
                .dropna(subset=['A.CD_GEOCODI', 'B.CD_GEOCODI'])

# Em caso de Desassociação em A e B, verificar se código permanece entre A e B
### Identifica desassociados que se sobrepõem à sua versão antiga
desassoc = intersecao.query('(`A.CLASS` == `B.CLASS` == "Desassociação") & `A.CD_GEOCODI` == `B.CD_GEOCODI`')
### Adiciona desassociados selecionados no grafo geral
for i, row in desassoc.iterrows():
    G_compat.add_edge(f"A.{row['A.CD_GEOCODI']}",
                      f"B.{row['B.CD_GEOCODI']}",
                      class_A=row['A.CLASS'],
                      class_B=row['B.CLASS'],
                      metodo='Herança (desassociado)')

# Seleciona apenas áreas que sofreram redesenho ou remoção de A para B
selected_class = ['Redesenho', 'Remoção']
intersecao = intersecao.query('`A.CLASS` in @selected_class')
# Calcular áreas resultantes
#intersecao['A.PCT_AREA'] = intersecao.area*100/intersecao['A.AREA_ORIGINAL']
intersecao['B.PCT_AREA'] = intersecao.area*100/intersecao['B.AREA_ORIGINAL']

# Registrar herança (>70% da área original de B em um único setor de A)
heranca = intersecao.query('`B.PCT_AREA` > 70')
for i, row in heranca.iterrows():
    G_compat.add_edge(f"A.{row['A.CD_GEOCODI']}",
                      f"B.{row['B.CD_GEOCODI']}",
                      class_A=row['A.CLASS'],
                      class_B=row['B.CLASS'],
                      metodo='Herança (área)')

### Sobreposição com buffers

O objetivo é tratar todos os casos restantes, atribuindo-os conforme a sobreposição geométrica entre malhas, permitindo para isso uma inconsistência de desenho entre as malhas correspondente à distância do buffer. Geometrias expúrias (razão área/perímetro baixa) são excluídas após interseção.

In [155]:
buffer_size = -25

In [156]:
# Selecionar aptos para sobreposição
### (usando todos exceto Manutenção, anteriormente filtrado para exceto Manutenção e Ajuste)
selecionados_A = gdf_A.query('CLASS != "Manutenção"')[['CD_GEOCODI', 'CLASS', 'geometry']]
selecionados_B = gdf_B.query('CLASS != "Manutenção"')[['CD_GEOCODI', 'CLASS', 'geometry']]

selecionados_A = selecionados_A.rename(columns={k:f'A.{k}' for k in selecionados_A.columns})
selecionados_B = selecionados_B.rename(columns={k:f'B.{k}' for k in selecionados_B.columns})

# Selecionar setores ainda não vinculados
list_isolados = list(nx.isolates(G_compat))

list_isolados_A =[i.split('.')[-1] for i in list_isolados if i.startswith('A.')]
isolados_A = gdf_A.query('CD_GEOCODI in @list_isolados_A')
isolados_A = isolados_A.rename(columns={k:f'A.{k}' for k in isolados_A.columns})

list_isolados_B =[i.split('.')[-1] for i in list_isolados if i.startswith('B.')]
isolados_B = gdf_B.query('CD_GEOCODI in @list_isolados_B')
isolados_B = isolados_B.rename(columns={k:f'B.{k}' for k in isolados_B.columns})

In [157]:
# Isolados de A para B
isolados_A_buffer = isolados_A.copy()
isolados_A_buffer['A.geometry'] = isolados_A_buffer['A.geometry'].buffer(buffer_size)
isolados_A_buffer = isolados_A_buffer.set_geometry('A.geometry', crs=UTMCRS)
### Restituir geometria para setores pequenos (buffer isolados A)
for i, row in isolados_A_buffer.iterrows():
    if row['A.geometry'].is_empty:
        isolados_A_buffer.loc[i, 'A.geometry'] = isolados_A.loc[i, 'A.geometry']

buffered_B = selecionados_B.copy()
buffered_B['B.geometry'] = buffered_B['B.geometry'].buffer(buffer_size)
### Restituir geometria para setores pequenos (buffer B)
for i, row in buffered_B.iterrows():
    if row['B.geometry'].is_empty:
        buffered_B.loc[i, 'B.geometry'] = selecionados_B.loc[i, 'B.geometry']
### Restituir coluna de geometria
buffered_B = buffered_B.set_geometry('B.geometry', crs=UTMCRS)
### Interseção
intersecao = gpd.overlay(isolados_A_buffer, buffered_B, how='union')\
                .dropna(subset=['A.CD_GEOCODI', 'B.CD_GEOCODI'])

### Remoção geometrias expúrias
#intersecao = intersecao[intersecao['geometry'].apply(geomNotEspuria)]

### Adição das arestas
for i, row in intersecao.iterrows():
    G_compat.add_edge(f"A.{row['A.CD_GEOCODI']}",
                      f"B.{row['B.CD_GEOCODI']}",
                      class_A=row['A.CLASS'],
                      class_B=row['B.CLASS'],
                      metodo='Buffer AB')

In [158]:
# Isolados de B para A
isolados_B_buffer = isolados_B.copy()
isolados_B_buffer['B.geometry'] = isolados_B_buffer['B.geometry'].buffer(buffer_size)
isolados_B_buffer = isolados_B_buffer.set_geometry('B.geometry', crs=UTMCRS)
### Restituir geometria para setores pequenos (buffer isolados B)
for i, row in isolados_B_buffer.iterrows():
    if row['B.geometry'].is_empty:
        isolados_B_buffer.loc[i, 'B.geometry'] = isolados_B.loc[i, 'B.geometry']

buffered_A = selecionados_A.copy()
buffered_A['A.geometry'] = buffered_A['A.geometry'].buffer(buffer_size)
### Restituir geometria para setores pequenos
for i, row in buffered_A.iterrows():
    if row['A.geometry'].is_empty:
        buffered_A.loc[i, 'A.geometry'] = selecionados_A.loc[i, 'A.geometry']
### Restituir coluna de geometria
buffered_A = buffered_A.set_geometry('A.geometry', crs=UTMCRS)
### Interseção
intersecao = gpd.overlay(isolados_B_buffer, buffered_A, how='union')\
                .dropna(subset=['A.CD_GEOCODI', 'B.CD_GEOCODI'])

### Remoção geometrias expúrias
#intersecao = intersecao[intersecao['geometry'].apply(geomNotEspuria)]

### Adição das arestas
for i, row in intersecao.iterrows():
    G_compat.add_edge(f"A.{row['A.CD_GEOCODI']}",
                      f"B.{row['B.CD_GEOCODI']}",
                      class_A=row['A.CLASS'],
                      class_B=row['B.CLASS'],
                      metodo='Buffer BA')

### Sobreposição forçada

In [159]:
buffer_sizes = [-10, -5, 0]

In [160]:
# Executa sobreposição com buffers crescentes até zero
for buffer_size in buffer_sizes:
    # Selecionar setores ainda não vinculados
    list_isolados = list(nx.isolates(G_compat))

    list_isolados_A = [i.split('.')[-1] for i in list_isolados if i.startswith('A.')]
    isolados_A = gdf_A.query('CD_GEOCODI in @list_isolados_A')
    isolados_A = isolados_A.rename(columns={k:f'A.{k}' for k in isolados_A.columns})

    list_isolados_B =[i.split('.')[-1] for i in list_isolados if i.startswith('B.')]
    isolados_B = gdf_B.query('CD_GEOCODI in @list_isolados_B')
    isolados_B = isolados_B.rename(columns={k:f'B.{k}' for k in isolados_B.columns})

    # Configurar geodataframes
    isolados_A = isolados_A.set_geometry('A.geometry', crs=UTMCRS)
    isolados_B = isolados_B.set_geometry('B.geometry', crs=UTMCRS)

    # Buffers
    buffered_A = selecionados_A.copy()
    buffered_A['A.geometry'] = buffered_A['A.geometry'].buffer(buffer_size)
    ### Restituir geometria para setores pequenos
    for i, row in buffered_A.iterrows():
        if row['A.geometry'].is_empty:
            buffered_A.loc[i, 'A.geometry'] = selecionados_A.loc[i, 'A.geometry']
    ### Restituir coluna de geometria
    buffered_A = buffered_A.set_geometry('A.geometry', crs=UTMCRS)

    buffered_B = selecionados_B.copy()
    buffered_B['B.geometry'] = buffered_B['B.geometry'].buffer(buffer_size)
    ### Restituir geometria para setores pequenos (buffer B)
    for i, row in buffered_B.iterrows():
        if row['B.geometry'].is_empty:
            buffered_B.loc[i, 'B.geometry'] = selecionados_B.loc[i, 'B.geometry']
    ### Restituir coluna de geometria
    buffered_B = buffered_B.set_geometry('B.geometry', crs=UTMCRS)

    # Isolados de A para B
    ### Interseção
    intersecao = gpd.overlay(isolados_A, buffered_B, how='union')\
                    .dropna(subset=['A.CD_GEOCODI', 'B.CD_GEOCODI'])

    ### Adição das arestas
    for i, row in intersecao.iterrows():
        G_compat.add_edge(f"A.{row['A.CD_GEOCODI']}",
                        f"B.{row['B.CD_GEOCODI']}",
                        class_A=row['A.CLASS'],
                        class_B=row['B.CLASS'],
                        metodo='Forçado AB')

    # Isolados de B para A
    ### Interseção
    intersecao = gpd.overlay(isolados_B, buffered_A, how='union')\
                    .dropna(subset=['A.CD_GEOCODI', 'B.CD_GEOCODI'])

    ### Adição das arestas
    for i, row in intersecao.iterrows():
        G_compat.add_edge(f"A.{row['A.CD_GEOCODI']}",
                        f"B.{row['B.CD_GEOCODI']}",
                        class_A=row['A.CLASS'],
                        class_B=row['B.CLASS'],
                        metodo='Forçado BA')

### Assersão de compatibilização completa

In [161]:
# Limpar arestas entre grupos
to_remove = []
for u, v, d in G_compat.edges(data=True):
    group_A = G_compat.nodes[u]['group']
    group_B = G_compat.nodes[v]['group']
    if group_A != group_B:
        to_remove.append([u, v])

for u, v in to_remove:
    # Remove as arestas entre grupos apenas de nós que tem vínculos válidos
    if not all([e in to_remove for e in G_compat.edges(u)]) and not all([e in to_remove for e in G_compat.edges(v)]):
        G_compat.remove_edge(u,v)

In [162]:
try:
    assert not list(nx.isolates(G_compat))
except AssertionError:
    print(len(list(nx.isolates(G_compat))), 'nós não conectados')

989 nós não conectados


### Codificação das componentes do grafo de compatibilização

In [163]:
componentes = [{'group':G_compat.nodes[list(G)[0]]['group'], 'nodes':list(G)}\
               for G in nx.connected_components(G_compat)]
increment_id = {k['group']:0 for k in componentes}

matriz_A = []
matriz_B = []
for c in componentes:
    increment_id[c['group']] += 1
    cod_c = f"{c['group']}{increment_id[c['group']]:05d}"

    c_A = [G_compat.nodes[i]['nome'] for i in c['nodes'] if G_compat.nodes[i]['malha']=='A']
    matriz_A.append({'CD_PERIMETRO':cod_c, 'CD_GEOCODI':c_A})
    c_B = [G_compat.nodes[i]['nome'] for i in c['nodes'] if G_compat.nodes[i]['malha']=='B']
    matriz_B.append({'CD_PERIMETRO':cod_c, 'CD_GEOCODI':c_B})

# Criação dos DataFrames finais
df_matriz_A = pd.DataFrame(matriz_A)
df_matriz_A = df_matriz_A.explode('CD_GEOCODI')
df_matriz_B = pd.DataFrame(matriz_B)
df_matriz_B = df_matriz_B.explode('CD_GEOCODI')

### Exportação de matrizes de compatibilidade

In [164]:
df_matriz_A[['CD_GEOCODI', 'CD_PERIMETRO']].to_csv(f'{nome_compat}/matriz_compat_{nome_A}.csv', sep='\t', index=False)
df_matriz_B[['CD_GEOCODI', 'CD_PERIMETRO']].to_csv(f'{nome_compat}/matriz_compat_{nome_B}.csv', sep='\t', index=False)

In [165]:
# Contagem de membros A dos perímetros
data_matriz_A = df_matriz_A.pivot_table(index='CD_PERIMETRO',
                                        values='CD_GEOCODI',
                                        aggfunc='count').reset_index()
data_matriz_A = data_matriz_A.rename(columns={'CD_GEOCODI':'membros_A'})
# Contagem de membros B dos perímetros
data_matriz_B = df_matriz_B.pivot_table(index='CD_PERIMETRO',
                                        values='CD_GEOCODI',
                                        aggfunc='count').reset_index()
data_matriz_B = data_matriz_B.rename(columns={'CD_GEOCODI':'membros_B'})
# Agregação dos dados
data_matrizes = data_matriz_A.merge(data_matriz_B, on='CD_PERIMETRO')
data_matrizes['membros'] = data_matrizes['membros_A'] + data_matrizes['membros_B']

### Geopackage de perímetros compatíveis

In [166]:
def removeHoles(geom, area_min=1):
    if isinstance(geom, Polygon):
        geom = MultiPolygon([geom])
    out_polys = []
    for part in geom.geoms:
        interiors = []
        for i in part.interiors:
            p = Polygon(i)
            if p.area > area_min:
                interiors.append(i)
        out_polys.append(Polygon(part.exterior.coords, holes=interiors))
    return MultiPolygon(out_polys) if len(out_polys)>1 else out_polys[0]

gdf_perim_compat = df_matriz_B.merge(gdf_B, on='CD_GEOCODI')
gdf_perim_compat = gdf_perim_compat.merge(data_matrizes, on='CD_PERIMETRO')
gdf_perim_compat = gpd.GeoDataFrame(gdf_perim_compat, geometry='geometry', crs=UTMCRS)
gdf_perim_compat = gdf_perim_compat[['CD_PERIMETRO', 'GROUP', 'membros', 'membros_A', 'membros_B', 'geometry']].dissolve(by='CD_PERIMETRO')
gdf_perim_compat = gdf_perim_compat.rename(columns={'GROUP':'CD_DIST'})
gdf_perim_compat['CD_MUN'] = gdf_perim_compat['CD_DIST'].apply(lambda x: x[:7])
gdf_perim_compat['geometry'] = gdf_perim_compat['geometry'].apply(removeHoles)
gdf_perim_compat.to_file(f'{nome_compat}/perimetros_compativeis.gpkg',
                         layer=f'{nome_A}-{nome_B}',
                         driver='GPKG')

### Exportação de geopackage com grafo de compatibilização

In [167]:
# Arestas
edge_data = []
for u, v in list(G_compat.edges):
    data_u = G_compat.nodes[u]
    data_u = {f"{data_u['malha']}.{k}":value for k, value in data_u.items()}
    data_v = G_compat.nodes[v]
    data_v = {f"{data_v['malha']}.{k}":value for k, value in data_v.items()}
    data_u.update(data_v)
    data_u.update(G_compat.edges[(u, v)])
    data_u['geometry'] = LineString([data_u['A.center'], data_u['B.center']])
    edge_data.append(data_u)

edge_gdf = gpd.GeoDataFrame(edge_data, geometry='geometry', crs=UTMCRS)
edge_gdf = edge_gdf[['A.nome', 'A.classe', 'B.nome', 'B.classe', 'class_A', 'class_B', 'metodo', 'geometry']]
edge_gdf.to_file(f'{nome_compat}/grafo_compatibilizacao.gpkg',
                    layer=f'{nome_A}-{nome_B}_edges',
                    driver='GPKG')

# Nós
node_data = [i for _, i in list(G_compat.nodes.data())]
for k in node_data:
    k['grau'] = len(G_compat[f"{k['malha']}.{k['nome']}"])
node_gdf = gpd.GeoDataFrame(node_data, geometry='center', crs=UTMCRS)
node_gdf = node_gdf[['nome', 'malha', 'classe', 'group', 'grau', 'center']]
node_gdf.to_file(f'{nome_compat}/grafo_compatibilizacao.gpkg',
                    layer=f'{nome_A}-{nome_B}_nodes',
                    driver='GPKG')